In [8]:
# let's take one dataset for example
from multilab.datasets import reuter
sentences, labels = reuter()

[nltk_data] Downloading package reuters to /Users/monk/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [24]:
# very few datapoints just to demonstrate how to use models

# preprocessing
from multilab.preprocess import Text_preprocessing
text_preprocessing = Text_preprocessing()
dataframe = text_preprocessing.labels_to_dataframe(sentences,labels)
preprocessded_dataset = text_preprocessing.initial_preprocess(dataframe, chunk_value = 5)
dataset_s , frequency_list_s = text_preprocessing.keep_labels(preprocessded_dataset,
                                                           keep_ratio=0.1)

# text preprocessing
slice_dataset = text_preprocessing.dataset_slice(dataset_s,ratio=0.01)

#tf-idf
all_sentences, all_labels = text_preprocessing.tf_idf(slice_dataset)

# split the dataset
X_train, X_test, y_train, y_test = text_preprocessing.split_dataset(all_sentences, all_labels)

lower_case done
punctuation removed
text cleaning done


In [27]:
from multilab.models import Base_models
Bm = Base_models(X_train, y_train, X_test,y_test)

# four base models

# BinaryRelevance
# ClassifierChain
# LabelPowerset
# Mlknn


In [33]:
print(Bm.BinaryRe())
print(Bm.classfier_chain())
print(Bm.powerset())
print(Bm.mlknn())

{'accuracy': 0.4074074074074074, 'f1_score': 0.4395604395604396}
{'accuracy': 0.4074074074074074, 'f1_score': 0.39285714285714285}
{'accuracy': 0.4074074074074074, 'f1_score': 0.39285714285714285}
{'accuracy': 0.2962962962962963, 'f1_score': 0.3902439024390244}


/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/monk/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 